# prox c-01 Script updated

__Notebooks starting with "prox c-" integrate new processes and methodologies for calculating proximity. Based on the develop of proximity 2024 (Script 21-proximity-analysis-mexico.py)__

Updated on 2024 05 06.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Step 0: Notebook/Script config

### Config - Base data required

In [4]:
# ------------------------------ BASE DATA REQUIRED ------------------------------
# Name of area of interest (Required)
city = 'Aguascalientes'
# Shape of the area of interest (Required directory)
aoi_dir = "../../../data/external/temporal_todocker/prox_aoi/aoi_ags.gpkg"
# Points of interest (Required directory)
# pois gdf must have a col named 'code' with a unique ID for each type of point of interest.
# This code will be searched in dicc parameters to be assigned to a source-->amenity-->eje.
pois_dir = "../../../data/external/temporal_todocker/prox_aoi/pois_ags.gpkg"

### Config - Analysis and output options

In [5]:
# ---------------------------- SCRIPT CONFIGURATION - ANALYSIS AND OUTPUT OPTIONS ----------------------------
# IMPORTANT NOTE:
# Network distance method used in function pois_time will always be 'length' since
# this notebook creates its own OSMnx Network ('time_min' is the result of pre-processing)
# Therefore, this script assumes pedestrian speed of 4km/hr

# Resolutions of hexgrid output (Required)
res_list = [8,9]
# Count available amenities at given time proximity (minutes)? (Required)
count_pois = (False,15) # Must pass a tupple containing a boolean (True or False) and time proximity of interest in minutes (Boolean,time)
# Save disk space by deleting used data that will not be used after? (Required)
save_space = False

# OPTIONAL 
pop_output = True
# Pop data by block file directory (Required if pop_output = True)
# Pop data is converted to hex data by using centroids.
pop_dir = "../../../data/external/temporal_todocker/prox_aoi/pop_gdf_ags.gpkg"
# List of columns with pop data. with total pop data (Required if pop_output = True)
# First item of list must be name of total population column in order to calculate density.
pop_columns = ['pobtot','pobfem','pobmas']
# Pop gdf index column (Required if pop_output = True)
pop_index_column = 'cvegeo'

### Config - Saving

In [6]:
# ---------------------------- SCRIPT CONFIGURATION - SAVING ----------------------------
# Save final output to database?
db_save = True
save_schema = 'prox_analysis'
nodes_save_table = 'nodesproximity_aoi'
hex_save_table = 'proximityanalysis_aoi'
# Test - (If testing, Script saves it ONLY locally. (Make sure directory exists)
test = True
nodes_local_save_dir = f"../../../data/processed/prox_aoi/test_{city}_script18_nodes.gpkg"
final_local_save_dir = f"../../../data/processed/prox_aoi/test_{city}_script18_hex.gpkg"

### Config - Pois structure

In [7]:
# ---------------------------- SCRIPT CONFIGURATION - POIS STRUCTURE ----------------------------
# PARAMETERS DICTIONARY (Required)
# Set the ejes, amenidades, sources and codes for analysis
        #{Eje (e):
        #            {Amenity (a):
        #                          {Sources (s):
        #                                           [Codes (c)]
        #                           }
        #             }
        #}
parameters = {'Escuelas':{'Preescolar':{'denue_preescolar':[611111, 611112]},
                        'Primaria':{'denue_primaria':[611121, 611122]},
                        'Secundaria':{'denue_secundaria':[611131, 611132]}
                        },
            'Servicios comunitarios':{'Salud':{'clues_primer_nivel':[8610]},
                                    'Guarderías':{'denue_guarderias':[624411, 624412]},
                                    'Asistencia social':{'denue_dif':[931610]}
                                    },
            'Comercio':{'Alimentos':{'denue_supermercado':[462111],
                                    'denue_abarrotes':[461110], 
                                    'denue_carnicerias': [461121, 461122, 461123],
                                    'sip_mercado':[4721]},
                        'Personal':{'denue_peluqueria':[812110]},
                        'Farmacias':{'denue_farmacias':[464111, 464112]},
                        'Hogar':{'denue_ferreteria_tlapaleria':[467111],
                                'denue_art_limpieza':[467115]},
                        'Complementarios':{'denue_ropa':[463211, 463212, 463213, 463215, 463216, 463218],
                                            'denue_calzado':[463310], 
                                            'denue_muebles':[466111, 466112, 466113, 466114],
                                            'denue_lavanderia':[812210],
                                            'denue_revistas_periodicos':[465313],
                                            'denue_pintura':[467113]}
                        },
            'Entretenimiento':{'Social':{'denue_restaurante_insitu':[722511, 722512, 722513, 722514, 722519],
                                        'denue_restaurante_llevar':[722516, 722518, 722517],
                                        'denue_bares':[722412],
                                        'denue_cafe':[722515]},
                                'Actividad física':{'sip_cancha':[93110],
                                                    'sip_unidad_deportiva':[93111],
                                                    'sip_espacio_publico':[9321],
                                                    'denue_parque_natural':[712190]},
                                'Cultural':{'denue_cines':[512130],
                                            'denue_museos':[712111, 712112]}
                                } 
            }

# WEIGHT DICTIONARY (Required)
# If need to measure nearest source for amenity, doesn't matter which, choose 'min'
# If need to measure access to all of the different sources in an amenity, choose 'max'
source_weight = {'Escuelas':{'Preescolar':'max', #There is only one source, no effect.
                            'Primaria':'max',  #There is only one source, no effect.
                            'Secundaria':'max'},  #There is only one source, no effect.
                'Servicios comunitarios':{'Salud':'max',  #There is only one source, no effect.
                                        'Guarderías':'max', #There is only one source, no effect.
                                        'Asistencia social':'max'},  #There is only one source, no effect.
                'Comercio':{'Alimentos':'min', # /////////////////////////////////////////////////////// Will choose min time to source because measuring access to nearest food source, doesn't matter which.
                            'Personal':'max', #There is only one source, no effect.
                            'Farmacias':'max', #There is only one source, no effect.
                            'Hogar':'min', # ////////////////////////////////////////////////////////// Will choose min time to source because measuring access to nearest source, doesn't matter which.
                            'Complementarios':'min'}, # /////////////////////////////////////////////// Will choose min time to source because measuring access to nearest source, doesn't matter which.
                'Entretenimiento':{'Social':'max', # ////////////////////////////////////////////////// Will choose max time to source because measuring access to all of them.
                                    'Actividad física':'min', # //////////////////////////////////////// Will choose min time to source because measuring access to nearest source, doesn't matter which.
                                    'Cultural':'min'} # //////////////////////////////////////////////// Will choose min time to source because measuring access to nearest source, doesn't matter which.
                }

### Config - Script run final configs

In [8]:
if test:
    db_save = False
    local_save = True
else:
    db_save = True
    local_save = False

## Main function

### Step 1 - Create OSMnx Network

In [74]:
##########################################################################################
# STEP 1: CREATE OSMNX NETWORK
# ------------------- This step downloads the osmnx network for the area of interest.

# Read area of interest (aoi)
aoi = gpd.read_file(aoi_dir)
aoi = aoi.to_crs("EPSG:4326")
print(f"--- Starting creation of osmnx network.")

# Download osmnx network (G, nodes and edges from bounding box of aoi)
G, nodes, edges = aup.create_osmnx_network(aoi,how='from_bbox')
print(f"--- Finished creating osmnx network.")

--- Starting creation of osmnx network.
Extracted min and max coordinates from the municipality. Polygon N:22.10033, S:21.62227, E-102.06451, W-102.59887.
Created OSMnx graph from bounding box.
Converted OSMnx graph to 60765 nodes and 143092 edges GeoDataFrame.
Filtered columns.
Column: osmid in nodes gdf, has a list in it, the column data was converted to string.
Column: lanes in nodes gdf, has a list in it, the column data was converted to string.
Column: name in nodes gdf, has a list in it, the column data was converted to string.
Column: highway in nodes gdf, has a list in it, the column data was converted to string.
Column: maxspeed in nodes gdf, has a list in it, the column data was converted to string.
Column: ref in nodes gdf, has a list in it, the column data was converted to string.
--- Finished creating osmnx network.


In [75]:
nodes.head(1)

,osmid,x,y,street_count,geometry
0,301189389,-102.342212,21.848544,4,POINT (-102.34221 21.84854)


In [76]:
edges.head(1)

,osmid,v,u,key,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service
0,713153965,1408187972,301189389,0,False,2,Calle Constitución,residential,<NA>,13.812,"LINESTRING (-102.34221 21.84854, -102.34219 21...",NaN,<NA>,NaN,NaN,NaN,NaN,NaN


### Step 2 - Analyse points of interest (to nodes)

#### Getting to pois_time

In [30]:
def pois_time_test(G, nodes, edges, pois, poi_name, prox_measure,count_pois=(False,0)):
    ##########################################################################################
    # STEP 1: NEAREST. 
    # Finds and assigns nearest node OSMID to each point of interest.
       
    # Defines projection for downloaded data
    pois = pois.set_crs("EPSG:4326")
    nodes = nodes.set_crs("EPSG:4326")
    edges = edges.set_crs("EPSG:4326")
    
    # In case there are no amenities of the type in the city, prevents it from crashing if len = 0
    if len(pois) == 0:
        nodes_time = nodes.copy()
    
        # Format
        nodes_time.reset_index(inplace=True)
        nodes_time = nodes_time.set_crs("EPSG:4326")
    
        # As no amenities were found, output columns are set to nan.
        nodes_time['time_'+poi_name] = np.nan # Time is set to np.nan.
        print(f"0 {poi_name} found. Time set to np.nan for all nodes.")
        if count_pois[0]: 
            nodes_time[f'{poi_name}_{count_pois[1]}min'] = np.nan # If requested pois_count, value is set to np.nan.
            print(f"0 {poi_name} found. Pois count set to nan for all nodes.")
            nodes_time = nodes_time[['osmid','time_'+poi_name,f'{poi_name}_{count_pois[1]}min','x','y','geometry']]
            return nodes_time
        else:
            nodes_time = nodes_time[['osmid','time_'+poi_name,'x','y','geometry']]
            return nodes_time
    
    else:
        ### Find nearest osmnx node for each DENUE point.
        nearest = aup.find_nearest(G, nodes, pois, return_distance= True)
        nearest = nearest.set_crs("EPSG:4326")
        print(f"Found and assigned nearest node osmid to each {poi_name}.")
        return nearest

In [14]:
### SIMPLIFICATION
parameters = {'Escuelas':{'Preescolar':{'denue_preescolar':[611111, 611112]}}}

In [31]:
##########################################################################################
# STEP 2: ANALYSE POINTS OF INTEREST
# ------------------- This step analysis times (and count of pois at given time proximity if requested) 
# ------------------- using function aup.pois_time. This step is based on script 21.
# ------------------- Main difference lies in how pois are read.

# Read points of interest (pois)
print(f"--- Loading all points of interest.")
pois = gpd.read_file(pois_dir)
pois = pois[['code','geometry']]
pois = pois.set_crs("EPSG:4326")
print(f"--- Loaded {len(pois)} points of interest.")

print(f"""
------------------------------------------------------------
STARTING source pois proximity to nodes analysis for {city}.""")
# PREP. FOR ANALYSIS
i = 0
# PREP. FOR ANALYSIS - List of columns used to deliver final format of Script part 1
all_analysis_cols = []

# SOURCE LOOP
for eje in parameters.keys():
    for amenity in parameters[eje]:
        for source in parameters[eje][amenity]:
            source_analysis_cols = []

            print(f"""
Analysing source {source}.""")
            
            # 2.1 --------------- SAVE ANALYSIS COLUMN NAMES
            # Source col to lists
            source_analysis_cols.append(source)
            all_analysis_cols.append(source)
            # If counting pois, create and append column 
            # count_col formated example: 'denue_preescolar_15min'
            if count_pois[0]:
                count_col = f'{source}_{count_pois[1]}min'
                source_analysis_cols.append(count_col)
                all_analysis_cols.append(count_col)

            # 2.2 --------------- GET POIS - Select source points of interest 
            # (concats all data corresponding to current source in source_pois)
            source_pois = gpd.GeoDataFrame()
            for code in parameters[eje][amenity][source]:
                code_pois = pois.loc[pois['code'] == code]
                print(f"Added {len(code_pois)} of code {code}.")
                source_pois = pd.concat([source_pois,code_pois])
            print(f"--- {source_pois.shape[0]} {source} pois. Analysing source pois proximity to nodes.")
            
            # 2.3 --------------- SOURCE ANALYSIS
            # Calculate time data from nodes to source
            source_nodes_time = pois_time_test(G, nodes, edges, source_pois, source, prox_measure='length', count_pois=count_pois)
            # Format
            #source_nodes_time.rename(columns={'time_'+source:source},inplace=True)
            #source_nodes_time = source_nodes_time[['osmid']+source_analysis_cols+['x','y','geometry']]

--- Loading all points of interest.
--- Loaded 20792 points of interest.

------------------------------------------------------------
STARTING source pois proximity to nodes analysis for Aguascalientes.

Analysing source denue_preescolar.
Added 113 of code 611111.
Added 193 of code 611112.
--- 306 denue_preescolar pois. Analysing source pois proximity to nodes.
Found and assigned nearest node osmid to each denue_preescolar.


In [34]:
nearest = source_nodes_time.copy()
nearest.head(1)

,code,geometry,osmid,distance_node
0,611111,POINT (-102.27464 21.90191),18381,16.377978


#### Accessing calculate_distance_nearest_poi from pois_time

In [51]:
def get_to_calculate_distance_nearest_poi(nodes,edges,nearest,prox_measure='length'):
    poi_name = source
    edges_test = edges.copy() 
    
    edges_test[prox_measure].fillna(edges_test[prox_measure].mean(),inplace=True)
    # If prox_measure = 'length', calculates time_min assuming walking speed = 4km/hr
    if prox_measure == 'length':
        edges_test['time_min'] = (edges_test['length']*60)/4000

    # 2.2 --------------- ELEMENTS NEEDED OUTSIDE THE ANALYSIS LOOP
    # The pois are divided by batches of 200 or 250 pois and analysed using the function calculate_distance_nearest_poi.

    # nodes_analysis is a nodes gdf (index reseted) used in the function aup.calculate_distance_nearest_poi.
    nodes_analysis = nodes.reset_index().copy()
    # nodes_time: int_gdf stores, processes time data within the loop and returns final gdf. (df_int, df_temp, df_min and nodes_distance in previous code versions)
    nodes_time = nodes.copy()

    # --------------- 2.3 PROCESSING DISTANCE
    print (f"Starting time analysis for {poi_name}.")

    # List of columns with output data by batch
    time_cols = []
    poiscount_cols = []

    # If possible, analyses by batches of 200 pois.
    if len(nearest) % 250:
        batch_size = len(nearest)/200
        for k in range(int(batch_size)+1):
            print(f"Starting range k = {k+1} of {int(batch_size)+1} for {poi_name}.")
            # Calculate
            source_process = nearest.iloc[int(200*k):int(200*(1+k))].copy()
            print(f"Process size: {len(source_process)} pois.")
            a = """
            nodes_distance_prep = aup.calculate_distance_nearest_poi(source_process, nodes_analysis, edges_test, poi_name, 'osmid', wght='time_min',count_pois=count_pois)

            # Extract from nodes_distance_prep the calculated time data
            batch_time_col = 'time_'+str(k)+poi_name
            time_cols.append(batch_time_col)
            nodes_time[batch_time_col] = nodes_distance_prep['dist_'+poi_name]

            # If requested, extract from nodes_distance_prep the calculated pois count
            if count_pois[0]:
                batch_poiscount_col = f'{poi_name}_{str(k)}_{count_pois[1]}min'
                poiscount_cols.append(batch_poiscount_col)
                nodes_time[batch_poiscount_col] = nodes_distance_prep[f'{poi_name}_{count_pois[1]}min']

        # After batch processing is over, find final output values for all batches.
        # For time data, apply the min function to time columns.
        nodes_time['time_'+poi_name] = nodes_time[time_cols].min(axis=1)
        # If requested, apply the sum function to pois_count columns. 
        if count_pois[0]:
            # Sum pois count
            nodes_time[f'{poi_name}_{count_pois[1]}min'] = nodes_time[poiscount_cols].sum(axis=1)"""
    
    # Else, analyses by batches of 250 pois.
    else:
        batch_size = len(nearest)/250
        for k in range(int(batch_size)+1):
            print(f"Starting range k = {k+1} of {int(batch_size)+1} for source {poi_name}.")
            # Calculate
            source_process = nearest.iloc[int(250*k):int(250*(1+k))].copy()
            print(f"Process size: {len(source_process)} pois.")
            a = """
            nodes_distance_prep = aup.calculate_distance_nearest_poi(source_process, nodes_analysis, edges_test, poi_name, 'osmid', wght='time_min',count_pois=count_pois)

            # Extract from nodes_distance_prep the calculated time data
            batch_time_col = 'time_'+str(k)+poi_name
            time_cols.append(batch_time_col)
            nodes_time[batch_time_col] = nodes_distance_prep['dist_'+poi_name]

            # If requested, extract from nodes_distance_prep the calculated pois count
            if count_pois[0]:
                batch_poiscount_col = f'{poi_name}_{str(k)}_{count_pois[1]}min'
                poiscount_cols.append(batch_poiscount_col)
                nodes_time[batch_poiscount_col] = nodes_distance_prep[f'{poi_name}_{count_pois[1]}min']

        # After batch processing is over, find final output values for all batches.
        # For time data, apply the min function to time columns.
        nodes_time['time_'+poi_name] = nodes_time[time_cols].min(axis=1)
        # If requested, apply the sum function to pois_count columns. 
        if count_pois[0]:
            # Sum pois count
            nodes_time[f'{poi_name}_{count_pois[1]}min'] = nodes_time[poiscount_cols].sum(axis=1)

    print(f"Finished time analysis for {poi_name}.")"""

    return edges_test,nodes_analysis, source_process

In [52]:
edges_test,nodes_analysis, source_process = get_to_calculate_distance_nearest_poi(nodes,edges,nearest,prox_measure='length')
source_process.head(1)

Starting time analysis for denue_preescolar.
Starting range k = 1 of 2 for denue_preescolar.
Process size: 200 pois.
Starting range k = 2 of 2 for denue_preescolar.
Process size: 106 pois.


,code,geometry,osmid,distance_node
200,611112,POINT (-102.24574 21.88898),24264,15.966903


In [53]:
edges_test.head(1)

,osmid,v,u,key,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,time_min
0,713153965,1408187972,301189389,0,False,2,Calle Constitución,residential,<NA>,13.812,"LINESTRING (-102.34221 21.84854, -102.34219 21...",NaN,<NA>,NaN,NaN,NaN,NaN,NaN,0.20718


In [54]:
nodes_analysis.head(1)

,index,osmid,x,y,street_count,geometry
0,0,301189389,-102.342212,21.848544,4,POINT (-102.34221 21.84854)


#### Accessing get_seeds from calculate_distance_nearest_poi

In [66]:
# Based on calculate_distance_nearest_poi
def get_to_get_seeds():
    gdf_f = source_process.copy()
    nodes_test2 = nodes_analysis.copy()
    edges_test2 = edges_test.copy()
    amenity_name = source
    column_name = 'osmid'
    wght = 'length'
    get_nearest_poi=(False, 'poi_id_column')
    count_pois = (False,15)
    max_distance=(0,'distance_node')

    # --- Required processing
    nodes = nodes.copy()
    edges = edges.copy()
    if max_distance[0] > 0:
        gdf_f = gdf_f.loc[gdf_f[max_distance[1]]<=max_distance[0]]
    g, weights, node_mapping = to_igraph(nodes,edges,wght=wght) #convert to igraph to run the calculations

    return weights

In [67]:
weights = get_to_get_seeds
weights

<function __main__.get_to_get_seeds()>

In [71]:
aoi_diss = aoi.dissolve()

G_hippo, nodes_hippo, edges_hippo = aup.graph_from_hippo(aoi_diss, schema='osmnx', edges_folder='edges_speed', nodes_folder='nodes')

In [72]:
nodes_hippo.head(1)

,x,y,street_count,geometry
osmid,,,,
272921360,-102.295073,21.872876,3,POINT (-102.29507 21.87288)


In [73]:
edges_hippo.head(1)

,,,osmid,oneway,lanes,name,highway,length,geometry,grade,grade_abs,access,tunnel,ref,maxspeed,bridge,junction,service,width,walkspeed,time_min
u,v,key,,,,,,,,,,,,,,,,,,,
3003502781,8424128014,0,296556921,True,1,Carretera Aguascalietes-San Marcos,primary,17.585,"LINESTRING (-102.20343 21.99742, -102.20327 21...",0.0,0.0,None,None,MEX 25,80,None,None,None,None,4.0,0.263775


# Continuation

In [ ]:
print(f"""
------------------------------------------------------------
STARTING source pois proximity to nodes analysis for {city}.""")
# PREP. FOR ANALYSIS
i = 0
# PREP. FOR ANALYSIS - List of columns used to deliver final format of Script part 1
all_analysis_cols = []

# SOURCE LOOP
for eje in parameters.keys():
    for amenity in parameters[eje]:
        for source in parameters[eje][amenity]:
            source_analysis_cols = []

            print(f"""
Analysing source {source}.""")
            
            # 2.1 --------------- SAVE ANALYSIS COLUMN NAMES
            # Source col to lists
            source_analysis_cols.append(source)
            all_analysis_cols.append(source)
            # If counting pois, create and append column 
            # count_col formated example: 'denue_preescolar_15min'
            if count_pois[0]:
                count_col = f'{source}_{count_pois[1]}min'
                source_analysis_cols.append(count_col)
                all_analysis_cols.append(count_col)

            # 2.2 --------------- GET POIS - Select source points of interest 
            # (concats all data corresponding to current source in source_pois)
            source_pois = gpd.GeoDataFrame()
            for code in parameters[eje][amenity][source]:
                code_pois = pois.loc[pois['code'] == code]
            source_pois = pd.concat([source_pois,code_pois])
            print(f"--- {source_pois.shape[0]} {source} pois. Analysing source pois proximity to nodes.")
            
            # 2.3 --------------- SOURCE ANALYSIS
            # Calculate time data from nodes to source
            source_nodes_time = aup.pois_time(G, nodes, edges, source_pois, source, prox_measure='length', count_pois=count_pois)
            # Format
            source_nodes_time.rename(columns={'time_'+source:source},inplace=True)
            source_nodes_time = source_nodes_time[['osmid']+source_analysis_cols+['x','y','geometry']]

            # 2.4 --------------- OUTPUT MERGE
            # Merge all sources time data in final output nodes gdf
            if i == 0: # For the first analysed source
                nodes_analysis = source_nodes_time.copy()
            else: # For the following
                nodes_analysis = pd.merge(nodes_analysis,source_nodes_time[['osmid']+source_analysis_cols],on='osmid')

            i = i+1

            print(f"--- FINISHED source {source}. Mean city time = {nodes_analysis[source].mean()}")
        
# 2.5 --------------- Final format for nodes
column_order = ['osmid'] + all_analysis_cols + ['x','y','geometry']
nodes_analysis = nodes_analysis[column_order]

if test:
    nodes_analysis.to_file(nodes_local_save_dir, driver='GPKG')
    print(f"--- Saved {city} nodes gdf locally.")

if save:
    nodes_analysis['city'] = city
    aup.gdf_to_db_slow(nodes_analysis, nodes_save_table, save_schema, if_exists='append')
    print(f"--- Saved {city} nodes gdf in database.")

print(f"""
------------------------------------------------------------
FINISHED source pois proximity to nodes analysis for {city}.""")